# Muller's Method for Complex Roots

In [ ]:
function newton(f, df, p0, n_max, rel_tol; verbose = true)
    
    converged = false;
    p = p0;
    p_old = p0;

    for i in 1:n_max

        p = p_old - f(p_old)/df(p_old);
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end

        if (i>1)
            if abs(p-p_old)/abs(p)< rel_tol
                converged = true;
                break
            end
        end

        p_old = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

In [ ]:
"""
    muller(f, p0, p1, p2, n_max, rel_tol; verbose = true)

Find a root of `f` using Muller's method (quadratic interpolation).

Arguments
- `f` : function (supports complex values)
- `p0,p1,p2` : three initial guesses (may be complex)
- `n_max` : maximum number of iterations
- `rel_tol` : relative tolerance for convergence: |p_{n+1}-p_n|/|p_{n+1}| < rel_tol
- `verbose` : if true, print iteration info (default: true)

Returns
- `p` : approximate root

Description
Muller's method fits a quadratic through (p0,f(p0)), (p1,f(p1)), (p2,f(p2)), solves that
quadratic for a root and chooses the root that gives a larger denominator to reduce
subtractive cancellation. The triple (p1,p2,p) is then used for the next iteration.
The method naturally handles complex arithmetic and can find complex roots starting
from real initial guesses.

Notes
- Convergence is typically superlinear (order ≈ 1.84) but not guaranteed for every
  function or choice of starting points.
- Be mindful of nearly zero denominators; using complex arithmetic (Julia's built-in)
  avoids issues with negative discriminant.
- If the algorithm fails to converge within `n_max` iterations, it returns the last
  iterate and prints an error message (unless `verbose` is false).

Example
```julia
p = muller(x->x^3 - 1, -1.0, 0.0, 1.5, 100, 1e-8)
```
"""
function muller(f, p0, p1, p2, n_max, rel_tol; verbose = true)
    
    converged = false;
    p = p2;

    for i in 1:n_max

        # solve for the constants a, b, and c
        c = f(p2);
        A = [(p0-p2)^2 p0-p2; (p1-p2)^2 p1-p2 ]; # builds matrix
        x = A\[f(p0)-c; f(p1)-c]; # solves for a and b
        a = x[1];
        b = x[2];
        
        # take the root with larger denominator
        if abs(b + sqrt(b^2-4*a*c))> abs(b - sqrt(b^2-4*a*c))
            p = p2 - 2*c/(b + sqrt(b^2-4*a*c));
        else
            p = p2 - 2*c/(b - sqrt(b^2-4*a*c));            
        end
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end
        
        if (i>1)
            if abs(p-p2)/abs(p)< rel_tol
                converged = true;
                break
            end
        end

        # update entries
        p0 = p1;
        p1 = p2;
        p2 = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

## Example 1
Find a complex root of
$$
x^3 -1
$$
With an appropriate starting guess, Newton can find the complex roots.  But the starting guess will need to be complex.

In Julia, `im` is the imaginary number $i$

In [ ]:
f = x-> x^3 - 1;
df = x->3*x^2;
p0 = 0. + 1. *im; # this leads to a complex root
# p0 =  2. + 1. *im;  # this leads to the real root
rel_tol = 1e-12;
n_max = 100;

p = newton(f, df, p0, n_max, rel_tol);

In [ ]:
@show (-1 + sqrt(3)*im)/2;

Try Muller's method:

In [ ]:
f = x-> x^3 - 1;
p0 = -1. + 1*im; # finds a complex root
p1 = 0. + 1*im; 
p2 = 0. + 2*im;

# p0 = -2.0 + 0 * im; # finds a real root
# p1 = -1.0 + 0 * im;
# p2 = 1.5 + 0.1 * im;

rel_tol = 1e-8;
n_max = 100;

p = muller(f, p0, p1, p2, n_max, rel_tol);

In [ ]:
p

## Example 2
Find a complex root of
$$
x^4 - 3x^3+x^2+x+1
$$

In [ ]:
f = x-> x^4-3*x^3 +x^2 +x + 1;
p0 = 0.5 + 0*im;
p1 = -0.5 + 0*im;
p2 = 0.0 + 0*im;

rel_tol = 1e-8;
n_max = 100;

p = muller(f, p0, p1, p2, n_max, rel_tol);

## Example 3
Find a  root of
$$
e^x + 1 = 0
$$
**NOTE** this is a not a polynomial.

In [ ]:
f = x-> exp(x) + 1;
p0 = 0.5 + 0*im;
p1 = -0.5 + 0*im;
p2 = 1.0 + 0*im;

rel_tol = 1e-8;
n_max = 100;

p = muller(f, p0, p1, p2, n_max, rel_tol);

In [ ]:
@show abs(f(-π * im));